In [151]:
# imports a library 'pandas', names it as 'pd'
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

We want to write a small function which returns the list of weeks in an appropriate format.  The function will take 'month' and 'yrs_back' as an input and then return a list of weeks going 'yrs_back' number of years back.  For instance, get_weeks_nums(3,2) gives all the weeks in march for two years back, starting from the most recent dataset.

In [48]:
def fix_time(num):
    if len(str(num)) == 2:
        return str(num)
    else:
        return '0'+str(num)

def get_week_nums(month,yrs_back):
    week_list=[]
    ref_date=datetime.date(2018,6,30)
    weeks_back=yrs_back*52
    for i in range(weeks_back):
        week_shift=datetime.timedelta(-7*i)
        new=ref_date+week_shift
        yr=str(new.year)[-2:]
        mt=fix_time(new.month)
        day=fix_time(new.day)
        string=yr+mt+day
        if int(mt)==month:
            week_list.append(int(string))
    return week_list

In [50]:
def scrape(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)

#I am going to focus on June for the last 3 years.
week_nums = get_week_nums(6,3)
df = scrape(week_nums)

In [2]:
#df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_180630.txt')

In [71]:
cols={x:x.strip() for x in df.columns}
df_ren=df.rename(columns=cols)
#df_small=df_ren[:1000]
df_small=df_ren

Next we want to extract a time stampe from a given row in the dataframe.  The following is an example of how to do this for a particular row.  In the next frame we do this for each row.

In [52]:
pd.to_datetime(df_small.iloc[1][6]+' '+df_small.iloc[1][7],infer_datetime_format=True)

Timestamp('2018-06-23 04:00:00')

In [73]:
def get_datetime(x):
    return pd.to_datetime(x[6]+' '+x[7],infer_datetime_format=True)

In [77]:
len(df_small)

2547963

In [81]:
df_small['datetime']=df_small.apply(get_datetime,axis=1)
#df_small.iloc[:100000].apply(get_datetime,axis=1)

In [82]:
df_small_clean=df_small.drop_duplicates(subset=['C/A','UNIT','SCP','STATION','LINENAME','datetime'])

At this point we should group by station and datetime **then** resample to make sure that we only have proper intervales of 4.

In [83]:
df_small_clean2=df_small_clean.groupby(['STATION','datetime'])[['EXITS']].sum()
df_small_clean3=df_small_clean2.reset_index()

Now we need to get diffs for each station, which means we need to find a list of stations, go by each one, sort by datetime, resample do a diff operation, etc.  The following takes in a dataframe like df_small_clean3 and a station and then gives back a dataframe for that particular station with the indices sorted by datetime.

In [84]:
def station_activity(df,station):
    df_station=df[df['STATION']==station]
    df_sort=df_station.sort_values(by=['datetime'])
    return df_sort

In [85]:
station_activity(df_small_clean3,'59 ST').head()

,STATION,datetime,EXITS
58360,59 ST,2016-05-28 00:00:00,955460529
58361,59 ST,2016-05-28 04:00:00,955461609
58362,59 ST,2016-05-28 08:00:00,955464213
58363,59 ST,2016-05-28 12:00:00,955471806
58364,59 ST,2016-05-28 16:00:00,955480958


OK, now for each station, apply a re-sample and then a diff map and drop the first element.  First, make a dataframe for each station, combine these in a list.

In writing the following definitions we are requring the seconds to be zero.  This is just a simple way to get rid of the wierd entries.

Note that in the next iteration we could instead do the following:  Resample both datasets hourly.  Simply take averages within each time window and then treat both datasets uniformally.  Still nead to drop wierd times.  

In [86]:
def good_time_mask(df):
    masks=[]
    for i in range(len(df)):
        test=(df['datetime'].iloc[i].hour%4==0)&(df['datetime'].iloc[i].second==0)        
        masks.append(test)
    return pd.Series(masks)

def bad_time_mask(df):
    masks=[]
    for i in range(len(df)):
        test=(df['datetime'].iloc[i].hour%4!=0)&(df['datetime'].iloc[i].second==0)
        masks.append(test)
    return pd.Series(masks)

We Hereby Declare:  All bad-time index sets will just be mapped to good time indices by hand.

In [87]:
#m3=good_time_mask(df_small_clean3)
#df_small_clean3[m3];

Below we demonstrate how the time deltas can be used to shift the timestamp.

In [60]:
ts=df_small_clean3.iloc[0][1]
t_delta=datetime.timedelta(0,0,0,0,0,2)
ts.second, ts-t_delta

(0, Timestamp('2018-06-23 00:00:00'))

In [ ]:
In the next step we write a function to shift the times on the rows of the bad times.

In [88]:
def shift_time(row):
    t_delta=datetime.timedelta(0,0,0,0,0,2)
    ts=row[1]
    row[1]=ts-t_delta
    return row

Let's check to see that shift_time works on a particular row.

In [89]:
shift_time(df_small_clean3.iloc[0])

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


STATION                    1 AV
datetime    2016-05-27 22:00:00
EXITS                3476159310
Name: 0, dtype: object

Finally, we write a function which takes in a dataframe and sets all the times to the proper intervales.  Basically, we break the dataframe into 'bad' and 'good' pieces and apply the shift to the bad pieces.

In [90]:
def bad_to_good(df):
    gm=good_time_mask(df)
    bm=bad_time_mask(df)
    g_df=df[gm]
    b_df=df[bm]
    b2g_df=b_df.apply(shift_time,axis=1)
    fin=pd.concat([g_df,b2g_df],ignore_index=True)
    return fin

In [91]:
df_small_clean4=bad_to_good(df_small_clean3)

Next, go by station and for each station, apply the diff operation, obtaining a list of dataframes containing the diff values.

In [123]:
stations=df_small_clean4['STATION'].unique()
station_diffs_dct={}
for st in stations:
    sa=station_activity(df_small_clean4,st)
    st_diffs=pd.DataFrame(sa['EXITS'].diff()).dropna()
    sa['EXITS']=st_diffs['EXITS']
    sa=sa.rename(columns={'EXITS':'diffs'})
    station_diffs_dct[st]=sa

Need to add a column for weekday and an hour, dropna, group by weekday and hour. Note that many of the cleaning steps below could be incorporated above when the dictionary is constructed.

In [311]:
#st_df=station_diffs_dct['59 ST'].dropna()
#st_df_filt=st_df[np.abs(st_df.diffs)<20000]
#st_df_filt['weekday']=st_df_filt['datetime'].apply(lambda x: x.weekday())
#st_df_filt['hour']=st_df_filt['datetime'].apply(lambda x: x.hour)
#st_df_clean2=st_df_filt.groupby(['weekday','hour'])['diffs'].mean()

In [124]:
dct_clean={}
for st in stations:
    st_df=station_diffs_dct[st].dropna()
    st_df_filt=st_df[np.abs(st_df.diffs)<20000]
    st_df_filt['weekday']=st_df_filt['datetime'].apply(lambda x: x.weekday())
    st_df_filt['hour']=st_df_filt['datetime'].apply(lambda x: x.hour)
    st_df_clean2=st_df_filt.groupby(['weekday','hour'])['diffs'].mean()
    dct_clean[st]=st_df_clean2

/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [146]:
dct_clean['103 ST'][:10]

weekday  hour
0        3        354.923077
         7       2859.583333
         11      3292.692308
         15      4609.000000
         19      5919.153846
         23      2457.153846
1        3        318.538462
         7       3544.846154
         11      3913.846154
         15      5212.000000
Name: diffs, dtype: float64

In [149]:
def activity_by_time(day,hour):
    #We must convert a 24hour of the day to one of 0,4,8...
    h2=((hour//4+1)*4)%24
    exits=[]
    for st in stations:
        try:
            leaving=dct_clean[st][day][h2]
            exits.append([st,leaving])
        except(KeyError,IndexError):
            pass
    sort_exits=sorted(exits,key=lambda x: x[1])[::-1]    
    return sort_exits

In [142]:
stations;

When inputting the time remember that Monday is zero, Tuesday is 1, etc.  The hour may be input as an integer and it internally converts to one of the time intervals [0,4], [4,8], etc.  The result appears in order from largest flux to smallest.

In [150]:
activity_by_time(3,10)

[['59 ST COLUMBUS', 19556.0],
 ['CHAMBERS ST', 18097.076923076922],
 ['50 ST', 15084.615384615385],
 ['JOURNAL SQUARE', 14687.0],
 ['BOWLING GREEN', 13463.153846153846],
 ['51 ST', 11969.181818181818],
 ['JAY ST-METROTEC', 9185.0],
 ['49 ST', 8690.615384615385],
 ['BOROUGH HALL', 8452.272727272728],
 ['BROOKLYN BRIDGE', 8038.461538461538],
 ['ATL AV-BARCLAY', 7503.384615384615],
 ['FLUSHING-MAIN', 6959.916666666667],
 ['WALL ST', 6828.428571428572],
 ['GRAND ST', 6779.0],
 ['168 ST', 6646.833333333333],
 ['5 AV/53 ST', 6116.692307692308],
 ['34 ST-PENN STA', 6049.4],
 ['5 AVE', 5838.692307692308],
 ['42 ST-PORT AUTH', 5708.6],
 ['COURT SQ', 5400.076923076923],
 ['JAMAICA CENTER', 5348.416666666667],
 ['1 AV', 5226.416666666667],
 ['LEXINGTON AV/63', 4864.538461538462],
 ['34 ST-HUDSON YD', 4811.384615384615],
 ['SPRING ST', 4796.846153846154],
 ['PRINCE ST', 4765.076923076923],
 ['BEDFORD AV', 4466.916666666667],
 ['RECTOR ST', 4456.153846153846],
 ['WORLD TRADE CTR', 4375.846153846154

In [136]:
dct_clean['40 ST LOWERY ST'][3][20]

3251.3846153846152